In [1]:
import gensim
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
def similar_word_rankings(a, n=100):
    sims = model.most_similar(a, topn = n)
    dic = dict(sims)
    similar_words = {k: v for k, v in dic.items() if not k.startswith("wiki")}
    similar_words = {k: similar_words[k] for k in list(similar_words)[:n]}
    return similar_words

def word_vectorizer(words):
    
    word_vecs = []
    
    for w in words:
        v = model.wv[w]
        word_vecs.append(v)
        
    vec = np.sum(word_vecs, axis = 0)
    
    return vec

def PCA_reduction(word_vec, dims = 2):
    
    pca = PCA(n_components=2)
    result = pca.fit_transform(word_vec)
    
    return result


def sim_matrix(sims):

    vecs = np.zeros((len(sims), 300))
    i = 0
    for k, v in sims.items():
        vec = word_vectorizer([k])
        vecs[i, :] = vec
        i = i+1
    return vecs

In [ ]:
#words should be an input
words = ['STAR_WARS', 'Islam', 'dog']
vocab = []
mats = []
for w in words:
    sims = similar_word_rankings(w, 10)
    for k, v in sims.items():
        vocab.append(k)
    sm = sim_matrix(sims)
    mats.append(sm)


final_mat = np.vstack((m for m in mats))

result = PCA_reduction(final_mat)

fig = plt.figure(figsize=(10,10), dpi = 80)
ax = fig.add_subplot(111)
ax.scatter(result[:, 0], result[:, 1])
for i, word in enumerate(vocab):
    ax.annotate(' '.join(word.split('_')), xy=(result[i, 0], result[i, 1]))
    
ax.set_xlim(-3, 3, 0.25)
ax.set_xlim(-3, 3, 0.25)

plt.show()